지금 db에서 null값인 데이터를 investing에 검색한 결과 크롤링

In [2]:
# import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time

from random import uniform, seed
seed(1)
import re

from bs4 import BeautifulSoup

In [3]:
ticker_list = pd.read_csv('./lctn_v2.csv', index_col=0)
ticker_list

,stock_code,v_fctr,h_fctr,lctn
2332,A,Market_Cap,BM_Ratio,7.0
2333,AA,Market_Cap,BM_Ratio,8.0
2334,AAC,Market_Cap,BM_Ratio,4.0
2335,AACG,Market_Cap,BM_Ratio,3.0
2336,AACI,Market_Cap,BM_Ratio,1.0
...,...,...,...,...
9154,ZY,Market_Cap,BM_Ratio,6.0
9155,ZYME,Market_Cap,BM_Ratio,5.0
9156,ZYNE,Market_Cap,BM_Ratio,3.0
9157,ZYXI,Market_Cap,BM_Ratio,4.0


In [4]:
ticker_list

,stock_code,v_fctr,h_fctr,lctn
2332,A,Market_Cap,BM_Ratio,7.0
2333,AA,Market_Cap,BM_Ratio,8.0
2334,AAC,Market_Cap,BM_Ratio,4.0
2335,AACG,Market_Cap,BM_Ratio,3.0
2336,AACI,Market_Cap,BM_Ratio,1.0
...,...,...,...,...
9154,ZY,Market_Cap,BM_Ratio,6.0
9155,ZYME,Market_Cap,BM_Ratio,5.0
9156,ZYNE,Market_Cap,BM_Ratio,3.0
9157,ZYXI,Market_Cap,BM_Ratio,4.0


In [115]:
def isin_print(ticker):
# ticker를 입력값으로 하여 이동할 url을 찾는다
    global prev_close, market_cap, shares, beta, price2book
    url= 'https://api.investing.com/api/search/v2/search?'
    data={
        "q": f"{ticker}"
    }
    header ={
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'origin': 'https://kr.investing.com',
    'referer': 'https://kr.investing.com/search/?q=950130',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    response = requests.get(url,headers=header,params=data)
    # url 추출
    if response.json()['quotes'] != []:
        detail_url = response.json()['quotes'][0]['url']

        # 찾은 url로 request한다.
        url='https://investing.com'+detail_url
        header ={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=header)

        # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        news_tit = soup.findAll("dl", {"class":"grid mobile:grid-cols-1 desktop:grid-cols-3 gap-x-5 mobile:p-2 mobile:bg-background-surface text-xs"})
        try:
            if news_tit != []:
                key_values = str(news_tit[0]).split('<div class="flex justify-between border-b py-2 desktop:py-0.5">')
                for i in range(len(key_values)):
                    key_value = key_values[i].split('</span><span>')[0]

                    if len(key_values[i].split('</span><span>'))>1: # 1인 경우는 의미없는 값
                        if key_values[i].split('</span><span>')[1][0] !='<':  # M, B와 같은 값 인식
                            key_value = key_value + key_values[i].split('</span><span>')[1][0] # M, B값 추가

                    if key_value.split('</dt>')[0][4:] == 'Prev. Close':
                        if key_value.find('span')!=-1:
                            prev_close = key_value.split('<span class="key-info_dd-numeric__2cYjc"><span>')[-1]
                        else:
                            prev_close = np.nan

                    elif key_value.split('</dt>')[0][4:] == 'Market Cap':
                        if key_value.find('span')!=-1:
                            market_cap = key_value.split('<span class="key-info_dd-numeric__2cYjc"><span>')[-1]
                        else:
                            market_cap = np.nan


                    elif key_value.split('</dt>')[0][4:] == 'Shares Outstanding':
                        if key_value.find('span')!=-1:
                            shares = key_value.split('<span class="key-info_dd-numeric__2cYjc"><span>')[-1]   
                        else:
                            shares = np.nan

                    elif key_value.split('</dt>')[0][4:] == 'Beta':
                        if key_value.find('span')!=-1:
                            beta = key_value.split('<span class="key-info_dd-numeric__2cYjc"><span>')[-1]
                        else:
                            beta = np.nan


                url='https://investing.com'+detail_url+'-ratios'
                header ={
                    "User-Agent": "Mozilla/5.0",
                    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                    'accept-encoding': 'gzip, deflate, br',
                    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'
                }
                response = requests.get(url,headers=header)

                # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
                html_doc = response.text

                if re.search('Price to Book', html_doc) != None:
                    if re.search('>\d.*?\d<',re.search('Price to Book.*?\d<', html_doc.replace("\n","ss"))[0]) != None:
                        price2book = (re.search('>\d.*?\d<',re.search('Price to Book.*?\d<', html_doc.replace("\n","ss"))[0])[0]).replace("<","").replace(">","")
                    else:
                        price2book = np.nan
                else:
                    price2book = np.nan
        except:
            pass
    return ticker, prev_close, market_cap, shares, beta, price2book

In [116]:
factor = pd.DataFrame(index = ticker_list['stock_code'])
for i, ticker in enumerate(ticker_list['stock_code']):
    result = isin_print(ticker) if isin_print(ticker) != None else (ticker, np.nan, np.nan, np.nan, np.nan, np.nan)
    if result == None: result = (ticker, np.nan, np.nan, np.nan, np.nan, np.nan)
    print(i, result)
    factor.loc[ticker, 'Prev_Close_Price'] = result[1]
    factor.loc[ticker, 'Market_Cap'] = result[2]
    factor.loc[ticker, 'Shares'] = result[3]
    factor.loc[ticker, 'Beta'] = result[4]
    factor.loc[ticker, 'Price_To_Book'] = result[5]

0 ('A', '127.44', '37.29B', '296,040,570', '1.07', '7.32')
1 ('AA', '39.01', '6.99B', '179,925,180', '2.33', '1.23')
2 ('AAC', '9.95', '1.24B', '125,000,000', '0.017', nan)
3 ('AACG', '1.89', '60.16M', '31,498,920', '1.16', '2.85')
4 ('AACI', '9.93', '205.54M', '20,709,500', nan, nan)
5 ('AADI', '13.61', '330M', '24,390,359', '1.59', '3.04')
6 ('AAIC', '2.73', '78.87M', '28,890,953', '1.47', '0.44')
7 ('AAL', '12.18', '7.83B', '649,845,927', '1.51', nan)
8 ('AAM$A', '1.008', '6.61B', '6,300,000,000', '0.691', '0.86')
9 ('AAMC', '14.66', '27M', '1,776,205', '1.39', nan)
10 ('AAME', '2.75', '56.71M', '20,398,497', '0.309', '0.52')
11 ('AAN', '9.5', '277.92M', '30,777,065', '0.945', '0.39')
12 ('AAOI', '2.78', '74.45M', '27,882,758', '1.8', '0.34')
13 ('AAON', '53.14', '2.86B', '53,071,178', '0.693', '5.82')
14 ('AAP', '161.32', '9.9B', '59,578,239', '1.24', '3.54')
15 ('AAPL', '140.09', '2.26T', '16,070,752,000', '1.23', '38.84')
16 ('AAQC', '9.83', '491.75M', '50,000,000', '0.013', nan)

44 ('ABUS', '1.91', '287.91M', '149,950,606', '2.23', '1.88')
45 ('ABVC', '0.88', '27.72M', '32,356,982', '0.304', '2.97')
46 ('AC', '39.75', '867.24M', '22,011,159', '1.17', '0.98')
47 ('ACA', '61.25', '2.95B', '48,357,261', '0.552', '1.48')
48 ('ACAB', '10.03', '376.13M', '37,500,000', nan, '1.27')
49 ('ACAD', '16.29', '2.61B', '161,843,185', '0.707', nan)
50 ('ACAH', '9.85', '424.78M', '43,125,000', '-0.04', nan)
51 ('ACAQ', '10.06', '323.03M', '32,110,000', nan, nan)
52 ('ACAX', '9.89', '111.31M', '11,255,000', nan, '1.27')
53 ('ACB', '1.81', '434.13M', '300,393,331', '2.99', '0.72')
54 ('ACBA', '10.5', '63.32M', '6,054,000', '-0.002', nan)
55 ('ACC', '2,384.75', '444.91B', '187,787,263', '0.59', '3.22')
56 ('ACCD', '11.48', '820.1M', '71,938,910', '1.63', '1.62')
57 ('ACCO', '4.97', '474.13M', '94,260,926', '1.84', '0.54')
58 ('ACDI', '10.16', '380.25M', '37,500,000', nan, nan)
59 ('ACEL', '8.45', '739.04M', '90,126,930', '1.42', '4.25')
60 ('ACER', '1.38', '20.61M', '16,100,000',

180 ('AGFY', '0.442', '10.21M', '26,678,477', '2.47', '0.13')
181 ('AGGR', '9.92', '384.79M', '38,750,000', '-0.014', nan)
182 ('AGI', '11.12', '4.11B', '392,207,718', '1.06', '1.13')
183 ('AGIL', '4.5', '224.1M', '48,193,369', '-0.421', '2.53')
184 ('AGIO', '27.09', '1.47B', '54,818,049', '1.18', '1.31')
185 ('AGL', '22.56', '8.86B', '410,936,377', '1.05', '8.02')
186 ('AGLE', '0.502', '29.38M', '61,172,510', '1.77', '0.35')
187 ('AGM', '101.12', '1.09B', '10,797,419', '0.951', '1.5')
188 ('AGM$G', '1.79', '41.23M', '24,254,842', '1.03', '1.65')
189 ('AGMH', '1.79', '41.23M', '24,254,842', '1.03', '1.65')
190 ('AGNC', '7.87', '3.89B', '528,169,382', '1.07', '0.63')
191 ('AGNCM', '7.87', '3.89B', '528,169,382', '1.07', '0.63')
192 ('AGNCN', '23.35', '3.89B', '528,169,382', '1.07', '0.63')
193 ('AGO', '50.25', '3.17B', '61,906,440', '1.05', '0.6')
194 ('AGR', '39.64', '15.24B', '386,624,231', '0.349', '0.79')
195 ('AGRI', '1.47', '23.43M', '15,622,686', '-0.178', '2.31')
196 ('AGRO', '8

205 ('AHH', '10.42', '927.28M', '88,340,840', '0.831', '1.99')
206 ('AHI', '0.53', nan, nan, '1.97', nan)
207 ('AHL$C', '434.4', '28.69B', '66,987,560', '0.6', '2.77')
208 ('AHL$E', '109.25', '1.48B', '12,921,000', '0.752', '1.21')
209 ('AHPA', '10.26', '294.98M', '28,750,000', '0.01', nan)
210 ('AHPI', '1.34', '5.1M', '4,013,537', '-2.65', '0.97')
211 ('AHT', '6.85', '231.83M', '34,901,478', '2.06', nan)
212 ('AI', '12.54', '1.32B', '108,481,529', '1.69', '1.35')
213 ('AIB', '10.06', '114.78M', '11,252,225', nan, '1.27')
214 ('AIC', '23.65', '689.63M', '28,890,953', '1.47', '0.44')
215 ('AIF', '12.02', '171.4M', '14,464,026', '0.582', nan)
216 ('AIG', '49.71', '36.83B', '760,416,143', '1.21', '0.82')
217 ('AIH', '1.61', '51.8M', '31,393,246', '-0.277', nan)
218 ('AIHS', '0.8511', '6.76M', '7,682,908', '0.621', '0.87')
219 ('AIKI', '6.92', '33.83M', '5,003,950', '1.33', '0.39')
220 ('AIM', '0.54', '25.89M', '48,049,300', '-0.437', '0.54')
221 ('AIMAU', '9.9', '104.49M', '10,554,500', n

277 ('ALL$G', '4.33', '182.92M', '43,243,489', '1.01', '1.27')
278 ('ALL$H', '35.22', '3.32B', '97,266,839', '1.36', '2.27')
279 ('ALL$I', '48.41', '2.75B', '57,071,967', '0.605', '1.03')
280 ('ALLE', '92.42', '8.06B', '87,838,044', '1.06', '10.38')
281 ('ALLK', '6.47', '543.94M', '84,726,080', '0.648', '2.44')
282 ('ALLO', '10.36', '1.57B', '143,806,825', '0.548', '1.95')
283 ('ALLR', '1.04', '9.23M', '10,260,157', nan, '0.92')
284 ('ALLT', '3.76', '139.4M', '36,587,444', '0.96', nan)
285 ('ALLY', '29.11', '8.87B', '308,529,835', '1.3', '0.76')
286 ('ALNA', '0.0727', '9.16M', '122,083,226', '1.13', '0.54')
287 ('ALNY', '195.81', '22.42B', '120,026,559', '0.737', '127.38')
288 ('ALOR', '10.04', '225.67M', '22,477,500', nan, '1.27')
289 ('ALOT', '11.58', '86.77M', '7,346,758', '0.676', '1.04')
290 ('ALPA', '9.75', '192.75M', '19,769,011', nan, nan)
291 ('ALPN', '6.83', '294.84M', '44,006,679', '1.59', '2.96')
292 ('ALPP', '0.54', '102.15M', '199,509,242', '9.19', '1.45')
293 ('ALR', '0.

...

6340 ('UTRS', '0.79', '22.22M', '28,859,990', nan, '0.72')
6341 ('UTSI', '3.79', '33.88M', '9,034,525', '0.838', '0.55')
6342 ('UTZ', '14.32', '2B', '140,161,835', '0.706', '2.84')
6343 ('UUU', '3.76', '8.42M', '2,312,887', '1.62', '1.86')
6344 ('UUUU', '6.75', '1.06B', '157,578,374', '1.7', '3.98')
6345 ('UVE', '8.56', '266.31M', '30,716,005', '0.961', '0.73')
6346 ('UVSP', '24.5', '726.19M', '29,353,021', '0.905', '0.95')
6347 ('UVV', '44.02', '1.09B', '24,602,321', '0.56', '0.82')
6348 ('UWMC', '3.26', '5.19B', '1,594,645,212', '1.02', '36.54')
6349 ('UXIN', '0.659', '289.79M', '469,069,580', '0.94', nan)
6350 ('V', '183.83', '376.9B', '2,068,830,746', '0.901', '11.55')
6351 ('VABK', '31.79', '167.99M', '5,326,271', '0.634', '1.25')
6352 ('VAC', '129.47', '5.04B', '39,285,977', '2.11', '1.84')
6353 ('VACC', '3.1', '110.16M', '37,216,162', '0.544', '0.43')
6354 ('VAL', '56.76', '4.19B', '75,175,547', '0.849', '3.68')
6355 ('VALE', '14.42', '65.47B', '4,591,261,979', '0.608', '1.83')


6472 ('VNTR', '0.86', '84.77M', '107,902,595', '1.39', '0.15')
6473 ('VOC', '7.88', '137.53M', '17,000,000', '1.52', '8.98')
6474 ('VOD', '11.3', '30.77B', '27,642,241,639', '0.662', '0.58')
6475 ('VOR', '3.93', '146.52M', '38,056,306', '0.732', '0.84')
6476 ('VORB', '2.86', '904.93M', '335,159,079', '1.18', '7.29')
6477 ('VOTE', '2.86', '904.93M', '335,159,079', '1.18', '7.29')
6478 ('VOXX', '7.19', '168.16M', '23,936,920', '1.14', '0.46')
6479 ('VOYA', '63.9', '6.27B', '97,918,656', '1.26', '1.39')
6480 ('VPCB', '9.93', '317.17M', '31,973,083', '0.018', nan)
6481 ('VPG', '31.48', '425.34M', '13,654,475', '1.28', '1.5')
6482 ('VPV', '9.66', '228.53M', '23,829,544', '0.26', '1.27')
6483 ('VQS', '0.87', '27.15M', '33,515,247', '-0.255', '1.14')
6484 ('VRA', '3.02', '94.54M', '31,098,169', '1.79', '0.34')
6485 ('VRAR', '5.25', '62.67M', '13,593,734', '1.21', '2.17')
6486 ('VRAY', '3.74', '693.27M', '181,009,565', '1.15', '5.55')
6487 ('VRCA', '2.43', '99.04M', '41,094,053', '2.17', '6.06

6605 ('WHG', '10', '80.65M', '8,044,556', '1.12', '0.7')
6606 ('WHLM', '4.025', '19.86M', '5,157,344', '0.698', '0.89')
6607 ('WHLR', '1.55', '14.33M', '9,938,436', '2.49', nan)
6608 ('WHLRD', '12.05', '14.33M', '9,938,436', '2.49', nan)
6609 ('WHLRP', '2.5', '14.33M', '9,938,436', '2.49', nan)
6610 ('WHR', '138.73', '7.69B', '54,508,056', '1.56', '1.91')
6611 ('WIA', '8.89', '204.3M', '23,322,256', '0.538', '1.27')
6612 ('WILC', '12.18', '176.94M', '13,867,017', '0.491', '1.1')
6613 ('WIMI', '1.43', '120.42M', '93,349,598', '0.617', '0.68')
6614 ('WINA', '217.01', '753.61M', '3,437,702', '0.757', nan)
6615 ('WING', '128.32', '3.92B', '29,914,368', '1.37', nan)
6616 ('WINN', '128.32', '3.92B', '29,914,368', '1.37', nan)
6617 ('WINT', '0.296', '9.04M', '30,627,878', '0.293', '0.47')
6618 ('WINV', '10.04', '144.33M', '14,375,000', nan, nan)
6619 ('WIRE', '123.89', '2.52B', '19,125,566', '1.27', '1.59')
6620 ('WISA', '0.58', '15.01M', '23,509,230', '-0.028', '1.6')
6621 ('WISH', '0.76', '

6738 ('XPOA', '0.44', '881M', '110,262,332', '-0.023', nan)
6739 ('XPOF', '19.8', '954.88M', '48,898,984', '1.66', nan)
6740 ('XPRO', '15.41', '1.67B', '108,702,849', '1.21', '1.31')
6741 ('XRAY', '28.41', '6.05B', '215,451,556', '0.926', '1.23')
6742 ('XRTX', '1.01', '18.3M', '17,989,687', '0.481', '2.06')
6743 ('XRX', '14.29', '2.19B', '155,569,686', '1.72', '0.56')
6744 ('XSPA', '0.7437', '69.08M', '94,278,118', '2.14', '0.97')
6745 ('XTLB', '1.15', '6.86M', '544,906,149', '1.03', '1.24')
6746 ('XTNT', '0.705', '73.92M', '101,981,250', '1.01', '2.63')
6747 ('XXII', '0.922', '196.81M', '214,784,741', '1.6', '1.98')
6748 ('XYF', '2.37', '126.71M', '54,852,990', '0.579', '0.21')
6749 ('XYL', '89.27', '16B', '180,181,765', '1.11', '5.01')
6750 ('Y', '843.1', '11.33B', '13,455,454', '0.598', '1.43')
6751 ('YALA', '3.24', '472.99M', '148,274,265', '1.62', '1.21')
6752 ('YCBD', '0.243', '15.31M', '60,555,595', '1.81', '0.33')
6753 ('YELL', '4.48', '228.71M', '51,626,827', '3.28', nan)
6754

6794 ('ZH', '1.15', '694.65M', '625,806,688', '1.18', '0.82')
6795 ('ZI', '44.4', '17.61B', '403,441,729', '1.62', '8.29')
6796 ('ZIG', '44.4', '17.61B', '403,441,729', '1.62', '8.29')
6797 ('ZIM', '24', '2.93B', '120,047,393', '1.5', '0.56')
6798 ('ZIMV', '9.03', '220.93M', '26,084,045', nan, '0.29')
6799 ('ZION', '50.37', '7.52B', '150,471,375', '1.26', '1.45')
6800 ('ZIONO', '25.1', '7.52B', '150,471,375', '1.26', '1.45')
6801 ('ZIONP', '19.28', '7.52B', '150,471,375', '1.26', '1.45')
6802 ('ZIP', '16.91', '1.94B', '113,884,083', '1.22', '12.96')
6803 ('ZIVO', '3.22', '31.32M', '9,419,660', '-0.066', '10.4')
6804 ('ZKIN', '0.74', '21.18M', '30,392,940', '1.76', '0.24')
6805 ('ZLAB', '33.03', '2.95B', '95,849,483', '1.06', '2.43')
6806 ('ZM', '76.59', '21.94B', '297,645,139', '-0.493', '3.66')
6807 ('ZNH', '28.92', '17.33B', '19,486,901,730', '1.23', '1.73')
6808 ('ZNTL', '21.58', '1.23B', '56,983,641', '1.88', '2.59')
6809 ('ZOM', '0.206', '196.48M', '979,949,668', '0.63', '0.74')
6

In [132]:
# Price To Book 값이 없을 때 긴 문자열이 들어오는 현상 발생
ptb_index = list(factor[factor.Price_To_Book.apply(lambda x: True if len(str(x))>100 else False)].index)

In [137]:
factor.loc[ptb_index, 'Price_To_Book'] = np.nan

In [139]:
factor.to_csv('ticker_all_values_clean.csv')

In [2]:
factor = pd.read_csv('ticker_all_values_clean.csv',thousands = ',')

In [257]:
factor

,stock_code,Prev_Close_Price,Market_Cap,Shares,Beta,Price_To_Book
0,A,127.440,37.29B,2.960406e+08,1.070,7.32
1,AA,39.010,6.99B,1.799252e+08,2.330,1.23
2,AAC,9.950,1.24B,1.250000e+08,0.017,NaN
3,AACG,1.890,60.16M,3.149892e+07,1.160,2.85
4,AACI,9.930,205.54M,2.070950e+07,NaN,NaN
...,...,...,...,...,...,...
6822,ZY,2.780,266.49M,1.040985e+08,2.020,1.13
6823,ZYME,5.900,368.32M,6.169439e+07,1.130,1.66
6824,ZYNE,0.741,30.4M,4.344560e+07,1.860,0.53
6825,ZYXI,8.800,334.52M,3.840666e+07,0.776,4.85


In [258]:
factor.Market_Cap.str[-1].unique()

array(['B', 'M', 'T', nan, 'K'], dtype=object)

In [260]:
factor['mul']=factor.Market_Cap.str[-1]
factor['mul'] = factor['mul'].apply(lambda x: 1000 if x =='K'else 1000000 if x=='M' else 1000000000 if x=='B' else 1000000000000)
factor['mul'] = factor['mul'].apply(lambda x: float(x))

factor['Market_Cap'] = factor.Market_Cap.str[:-1]
factor['Market_Cap'] = factor['Market_Cap'].apply(lambda x: float(x))

factor['Market_Cap']  = factor['Market_Cap'] * factor['mul']

factor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6827 entries, 0 to 6826
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stock_code        6827 non-null   object 
 1   Prev_Close_Price  6825 non-null   float64
 2   Market_Cap        6776 non-null   float64
 3   Shares            6776 non-null   float64
 4   Beta              6292 non-null   float64
 5   Price_To_Book     5671 non-null   float64
 6   mul               6827 non-null   float64
dtypes: float64(6), object(1)
memory usage: 373.5+ KB


In [266]:
factor['BPS']=factor.Prev_Close_Price / factor.Price_To_Book

In [268]:
factor['BM_Ratio']=factor.Shares * factor.BPS / factor.Market_Cap

In [269]:
factor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6827 entries, 0 to 6826
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stock_code        6827 non-null   object 
 1   Prev_Close_Price  6825 non-null   float64
 2   Market_Cap        6776 non-null   float64
 3   Shares            6776 non-null   float64
 4   Beta              6292 non-null   float64
 5   Price_To_Book     5671 non-null   float64
 6   mul               6827 non-null   float64
 7   BPS               5671 non-null   float64
 8   BM_Ratio          5658 non-null   float64
dtypes: float64(8), object(1)
memory usage: 480.1+ KB


In [ ]:
factor_BM = factor.BM_Ratio.reset_index().drop('index',axis=1)
factor_BM.columns = ['Values']

In [277]:
factor_M = factor.Market_Cap.reset_index().drop('index',axis=1)
factor_M.columns = ['Values']

factor_B = factor.Beta.reset_index().drop('index',axis=1)
factor_B.columns = ['Values']

In [282]:
factor_BM = pd.concat([factor.stock_code , factor_BM],axis=1)
factor_M = pd.concat([factor.stock_code , factor_M],axis=1)
factor_B = pd.concat([factor.stock_code , factor_B],axis=1)

In [290]:
factor_BM['Factor'] = 'BM_Ratio'
factor_M['Factor'] = 'Market_Cap'
factor_B['Factor'] = 'Beta'

In [292]:
factor_all = pd.concat([factor_M,factor_BM,factor_B],axis=0).reset_index().drop('index', axis=1)

In [293]:
factor_all.to_csv('factor_all.csv')

BM_ratio = share X bps / market cap

BPS = Prev_Close_Price / Price_To_Book

전처리 해야할 부분
- Prev_Close_Price
    - 1000 이상의 값 1,000 표기
- Market_Cap
    - 들어가면 안되는 % 표기
    - 1000000 이상의 값 K,M,B 표기
- Shares
    - 들어가면 안되는 % 표기
    - 1000 이상의 값 1,000 표기
- Beta
    - 1000 이상의 값 1,000 표기
- Price_To_Book
    - 정규표현 잘못되서 들어간 string